# Advent of Code: Day 7

--- Day 7: Camel Cards ---

Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

- Five of a kind, where all five cards have the same label: AAAAA
- Four of a kind, where four cards have the same label and one card has a different label: AA8AA
- Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
- Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
- Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
- One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
- High card, where all cards' labels are distinct: 23456

Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:

```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```

This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

- 32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
- KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
- T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.

Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [ ]:
from __future__ import annotations
from collections import Counter
from dataclasses import dataclass
from enum import IntEnum
from functools import total_ordering

In [ ]:
class HandType(IntEnum):
    HIGH_CARD       = 1
    ONE_PAIR        = 2
    TWO_PAIR        = 3
    THREE_OF_A_KIND = 4
    FULL_HOUSE      = 5
    FOUR_OF_A_KIND  = 6
    FIVE_OF_A_KIND  = 7
    
@dataclass
@total_ordering
class Hand:
    cards: list[str]
    bid: int

    def get_type(self):
        counter = Counter(self.cards)
        counter_values = tuple(sorted(counter.values(), reverse=True))
        # print(self.cards, counter, counter_values)
        if counter_values == (5,):
            hand_type = HandType.FIVE_OF_A_KIND
        elif counter_values == (4, 1):
            hand_type = HandType.FOUR_OF_A_KIND
        elif counter_values == (3, 2):
            hand_type = HandType.FULL_HOUSE
        elif counter_values == (3, 1, 1):
            hand_type = HandType.THREE_OF_A_KIND
        elif counter_values == (2, 2, 1):
            hand_type = HandType.TWO_PAIR
        elif counter_values == (2, 1, 1, 1):
            hand_type = HandType.ONE_PAIR
        elif counter_values == (1, 1, 1, 1, 1):
            hand_type = HandType.HIGH_CARD
        else:
            raise ValueError(f'Unhandled case: cards: {self.cards}, counter: {counter}, counter_values: {counter_values}')
        return hand_type

    def __eq__(self, other):
        return self.get_type() == other.get_type() and tuple(self.cards) == tuple(other.cards)

    def __lt__(self, other):
        if self.get_type() < other.get_type():
            return True
        elif self.get_type() > other.get_type():
            return False
        else:
            alphabet = list(reversed('AKQJT98765432'))
            this_card_ordering = tuple([alphabet.index(card) for card in self.cards])
            other_card_ordering = tuple([alphabet.index(card) for card in other.cards])
            # print('self: ', self.cards, this_card_ordering)
            # print('other:', other.cards, other_card_ordering)
            return this_card_ordering < other_card_ordering
    
    @staticmethod
    def create_from_line(s: str) -> Hand:
        cards_str, bid = s.split()
        bid = int(bid)
        cards = [ch for ch in cards_str]
        return Hand(cards=cards, bid=bid)


def parse_input(lines: list[str]) -> list[Hand]:
    hands: list[Hand] = []
    for line in lines:
        hand: Hand = Hand.create_from_line(line)
        hands.append(hand)
    return hands


def get_input() -> list[str]:
    with open('20231207.txt', 'r') as f:
        return [line.strip() for line in f.readlines()]


def get_input_small() -> list[str]:
    return [
        '32T3K 765',
        'T55J5 684',
        'KK677 28',
        'KTJJT 220',
        'QQQJA 483',
    ]


def solution_part1():
    # input_lines: list[str] = get_input_small()
    input_lines: list[str] = get_input()
    hands: List[Hand] = parse_input(input_lines)
    # for hand in hands:
    #     print(hand.get_type())
    winnings: list[int] = []
    for idx, hand in enumerate(sorted(hands, reverse=False)):
        rank: int = idx + 1
        win = rank * hand.bid
        # print('rank', rank, 'hand:', 'win:', win, hand, hand.get_type())
        winnings.append(win)
    return sum(winnings)


solution_part1()

In [ ]:
hand: Hand = Hand.create_from_line('QQQJA 483')
print(hand)
counter = Counter(hand.cards)
print(counter)
tuple(sorted(counter.values(), reverse=True)) == (3, 1, 1)

In [ ]:
hand: Hand = Hand.create_from_line('QQQJA 483')
print(hand)
alphabet = list(reversed('AKQJT98765432'))
tuple([alphabet.index(card) for card in hand.cards])

In [ ]:
tuple([alphabet.index(card) for card in 'KK677'])

In [ ]:
tuple([alphabet.index(card) for card in 'KK677']) > tuple([alphabet.index(card) for card in 'KTJJT'])

## Part two

--- Part Two ---
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

```
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
```

- 32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
- KK677 is now the only two pair, making it the second-weakest hand.
- T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.

With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [ ]:
@dataclass
@total_ordering
class WildHand:
    cards: list[str]
    bid: int

    def get_type(self):
        wild_count = sum([card == 'J' for card in self.cards])
        counter = Counter([card for card in self.cards if card != 'J'])
        # print(self.cards, 'wild count:', wild_count)
        counter_values = tuple(sorted(counter.values(), reverse=True))
        # print(self.cards, counter, counter_values)
        try:
            if len(counter_values) == 0:
                counter_values = (wild_count,)
            else:
                counter_values = tuple([counter_values[0] + wild_count] + list(counter_values[1:]))
        except IndexError as e:
            print('IndexError: cards:', self.cards)
            raise e
        # print(self.cards, counter, counter_values)
        if counter_values == (5,):
            hand_type = HandType.FIVE_OF_A_KIND
        elif counter_values == (4, 1):
            hand_type = HandType.FOUR_OF_A_KIND
        elif counter_values == (3, 2):
            hand_type = HandType.FULL_HOUSE
        elif counter_values == (3, 1, 1):
            hand_type = HandType.THREE_OF_A_KIND
        elif counter_values == (2, 2, 1):
            hand_type = HandType.TWO_PAIR
        elif counter_values == (2, 1, 1, 1):
            hand_type = HandType.ONE_PAIR
        elif counter_values == (1, 1, 1, 1, 1):
            hand_type = HandType.HIGH_CARD
        else:
            raise ValueError(f'Unhandled case: cards: {self.cards}, counter: {counter}, counter_values: {counter_values}')
        return hand_type
    

    def __eq__(self, other):
        return self.get_type() == other.get_type() and tuple(self.cards) == tuple(other.cards)

    def __lt__(self, other):
        if self.get_type() < other.get_type():
            return True
        elif self.get_type() > other.get_type():
            return False
        else:
            alphabet = list(reversed('AKQT98765432J'))
            this_card_ordering = tuple([alphabet.index(card) for card in self.cards])
            other_card_ordering = tuple([alphabet.index(card) for card in other.cards])
            # print('self: ', self.cards, this_card_ordering)
            # print('other:', other.cards, other_card_ordering)
            return this_card_ordering < other_card_ordering
    
    @staticmethod
    def create_from_line(s: str) -> WildHand:
        cards_str, bid = s.split()
        bid = int(bid)
        cards = [ch for ch in cards_str]
        return WildHand(cards=cards, bid=bid)


def parse_input_part2(lines: list[str]) -> list[WildHand]:
    hands: list[WildHand] = []
    for line in lines:
        hand: WildHand = WildHand.create_from_line(line)
        hands.append(hand)
    return hands


def solution_part2():
    # input_lines: list[str] = get_input_small()
    input_lines: list[str] = get_input()
    hands: List[WildHand] = parse_input_part2(input_lines)
    # for hand in hands:
    #     print(hand.get_type())
    winnings: list[int] = []
    for idx, hand in enumerate(sorted(hands, reverse=False)):
        rank: int = idx + 1
        win = rank * hand.bid
        print('rank', rank, 'hand:', hand, 'win:', win, hand.get_type())
        winnings.append(win)
    return sum(winnings)


solution_part2()